In [1]:
from keras.layers import Input, Lambda,RepeatVector,Dense,Reshape,Dropout
from keras.models import Model
from keras import backend as K
import keras

def dimention_extender(arg, n=512):
    from keras import backend as K
    a= K.reshape(arg,(K.shape(arg)[0],1))
    return K.repeat_elements(a, n, 1)

def words_extender(args):
    from keras import backend as K
    a=RepeatVector(K.shape(args[0])[0])(args[1])
    #print(K.shape(a))
    #print(K.shape(args[1]))
    b = K.cast(args[0], dtype='float32')
    return K.dot(a,b)

def padding_extender(arg):
    from keras import backend as K
    a = K.zeros_like(arg) - K.ones_like(arg) - K.ones_like(arg)
    return K.dot(K.cast(arg,dtype='float32'),a)

def kols_extender(arg):
    from keras import backend as K
    a = K.arange(K.shape(arg)[0])
    a = K.reshape(a,(K.shape(a)[0],1))
    out = K.dot(a,arg)#return K.dot(a,arg) #
    return K.expand_dims(out, axis=2) #K.reshape(out, (K.shape(out)[0], K.shape(out)[1],1))

def output_filter(args):
    from keras import backend as K
    a = dimention_extender(arg[0], n=100)
    return K.dot(a,args[1])

def output_concatenator(args):
    from keras import backend as K
    a = K.zeros_like(args[0])
    a += args[0]+args[1]+args[2]
    return a


def get_position_encoding(arg, min_timescale=1.0, max_timescale=1.0e4):
    import math
    from keras import backend as K
    
    length=K.shape(arg)
    hidden_size=100
    
    position = K.cast(K.range(length), K.float32)
    num_timescales = hidden_size // 2
    log_timescale_increment = (
          math.log(float(max_timescale) / float(min_timescale)) /
          (K.cast(num_timescales, tf.float32) - 1))
    inv_timescales = min_timescale * tf.exp(
          tf.cast(K.range(num_timescales), K.float32) * -log_timescale_increment)
    scaled_time = K.expand_dims(position, 1) * K.expand_dims(inv_timescales, 0)
    signal = K.concat([K.sin(scaled_time), K.cos(scaled_time)], axis=1)
    return signal + arg

def reshape(arg):
    from keras import backend as K
    return K.reshape(arg,(K.shape(arg)[0]*K.shape(arg)[1],K.shape(arg)[2]))

Using TensorFlow backend.
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu

In [2]:
from keras_transformer import *

import numpy as np
from keras_layer_normalization import LayerNormalization
from keras_multi_head import MultiHeadAttention
from keras_position_wise_feed_forward import FeedForward
from keras_pos_embd import TrigPosEmbedding
from keras_embed_sim import EmbeddingRet, EmbeddingSim
from keras.models import load_model


def get_m(token_num,
              embed_dim,
              encoder_num,
              decoder_num,
              head_num,
              hidden_dim,
              attention_activation=None,
              feed_forward_activation='relu',
              dropout_rate=0.0,
              embed_weights =None,
              embed_trainable=None,
              trainable=True,
              use_adapter=False,
              adapter_units=None,
              adapter_activation='relu'):
    """Get full model without compilation.
    :param token_num: Number of distinct tokens.
    :param embed_dim: Dimension of token embedding.
    :param encoder_num: Number of encoder components.
    :param decoder_num: Number of decoder components.
    :param head_num: Number of heads in multi-head self-attention.
    :param hidden_dim: Hidden dimension of feed forward layer.
    :param attention_activation: Activation for multi-head self-attention.
    :param feed_forward_activation: Activation for feed-forward layer.
    :param dropout_rate: Dropout rate.
    :param use_same_embed: Whether to use the same token embedding layer. `token_num`, `embed_weights` and
                           `embed_trainable` should be lists of two elements if it is False.
    :param embed_weights: Initial weights of token embedding.
    :param embed_trainable: Whether the token embedding is trainable. It will automatically set to False if the given
                            value is None when embedding weights has been provided.
    :param trainable: Whether the layers are trainable.
    :param use_adapter: Whether to use feed-forward adapters before each residual connections.
    :param adapter_units: The dimension of the first transformation in feed-forward adapter.
    :param adapter_activation: The activation after the first transformation in feed-forward adapter.
    :return: Keras model.
    """
    decoder_token_num = token_num

    decoder_embed_weights = embed_weights

    if decoder_embed_weights is not None:
        decoder_embed_weights = [decoder_embed_weights]

    decoder_embed_trainable = embed_trainable

    if decoder_embed_trainable is None:
        decoder_embed_trainable = decoder_embed_weights is None


    decoder_embed_layer = EmbeddingRet(
        input_dim=decoder_token_num,
        output_dim=embed_dim,
        mask_zero=True,
        weights=decoder_embed_weights,
        trainable=decoder_embed_trainable,
        name='Decoder-Token-Embedding',
    )
    
    
    vector = Input(shape=(512,), name='Vectors-Input',dtype='float32')
    pos_words = Input(shape=(None,), name='Position-Words',dtype='int32')
    pos_pads = Input(shape=(None,), name='Position-Paddings',dtype='int32')
    pos_end = Input(shape=(None,), name='Position-End',dtype='int32')
    
    pos_words_dim_ex = Lambda(dimention_extender, name='Position-Words-Dimentions-Extender')(pos_words)
    pos_pads_dim_ex = Lambda(dimention_extender, name='Position-Paddings-Dimentions-Extender',arguments={'n':100})(pos_pads)
    pos_ens_ex = Lambda(dimention_extender, name='Position-End-Dimentions-Extender',arguments={'n':100})(pos_end)    
    
    pos_pads_ex = Lambda(padding_extender, name='Padding-Extender')(pos_pads_dim_ex)
    pos_vec_ex = Lambda(words_extender, name='Words-Extender')([pos_words_dim_ex,vector])
    pos_word_ex = Lambda(kols_extender, name='Numbers-Extender')(pos_words)
    
    pos_vec_ex_r = Lambda(reshape, name='Reshape_')(pos_vec_ex)
    pos_word_ex_r = Lambda(reshape, name='Reshape')(pos_word_ex)
    print(K.shape(pos_vec_ex))
    print(K.shape(pos_vec_ex_r))
    encoder_shifter_model = load_model('D:/112/decoder/models/sen_to_word2_0.h5') 
    garbedge, encoder_shifter_layer  = encoder_shifter_model([pos_vec_ex_r,pos_word_ex_r])
    #encoder_shifter_layer.trainable = False
    encoded_words= encoder_shifter_layer.outputs[1]
    
    encoded_words_filter = Lambda(output_filter, name='Encoded_Words_Filter')([pos_words,encoded_words])
    encoder_concatenator = Lambda(output_concatenator, name='Encoder_Concatenator')([encoded_words_filter,pos_pads_ex,pos_ens_ex])
    pos_wised_encoder = Lambda(get_position_encoding, name='Encoder_Concatenator')(encoder_concatenator)
    
    
    
    encoded_layer = get_encoders(
        encoder_num=encoder_num,
        input_layer=pos_wised_encoder,
        head_num=head_num,
        hidden_dim=hidden_dim,
        attention_activation=attention_activation,
        feed_forward_activation=feed_forward_activation,
        dropout_rate=dropout_rate,
        trainable=trainable,
        use_adapter=use_adapter,
        adapter_units=adapter_units,
        adapter_activation=adapter_activation,
    )
    
    
    decoder_input = keras.layers.Input(shape=(None,), name='Decoder-Input') 
    decoder_embed, decoder_embed_weights = decoder_embed_layer(decoder_input)
    decoder_embed = TrigPosEmbedding(
        mode=TrigPosEmbedding.MODE_ADD,
        name='Decoder-Embedding',
    )(decoder_embed)
    decoded_layer = get_decoders(
        decoder_num=decoder_num,
        input_layer=decoder_embed,
        encoded_layer=encoded_layer,
        head_num=head_num,
        hidden_dim=hidden_dim,
        attention_activation=attention_activation,
        feed_forward_activation=feed_forward_activation,
        dropout_rate=dropout_rate,
        trainable=trainable,
        use_adapter=use_adapter,
        adapter_units=adapter_units,
        adapter_activation=adapter_activation,
    )
    dense_layer = EmbeddingSim(
        trainable=trainable,
        name='Output',
    )([decoded_layer, decoder_embed_weights])
    return keras.models.Model(inputs=[vector,n_words,t_words,decoder_input], outputs=dense_layer)

In [3]:
from keras.utils import to_categorical

def piecesextender(data):
    kol= np.zeros(shape=(data.shape[0]))
    for i in range(data.shape[0]):
        kol[i]=np.count_nonzero(data[i])
    kol = kol.astype(np.int)
    out = np.zeros(shape=(np.sum(kol),512))
    buf = 0
    for g in range(data.shape[0]):        
        for k in range(1,kol[g]-1):
            out[buf][0]=3000
            out[buf][1:513-k]+=data[g][k:]
            buf+=1
    return out, kol

def vectorsextender(data,kol):
    out = np.zeros(shape=(np.sum(kol)-2*data.shape[0],512))
    buf=0
    for g in range(data.shape[0]):
        for k in range(kol[g]-2):
            out[buf]+=data[g]
            buf+=1
    return out

def kolextender(kol):
    out = np.zeros(shape=(np.sum(kol)-2*kol.shape[0],512))
    out2 = np.zeros(shape=(np.sum(kol)-2*kol.shape[0],512))
    out3 = np.zeros(shape=(np.sum(kol)-2*kol.shape[0],512))
    buf = 0
    for g in range(kol.shape[0]):
        for k in range(1, kol[g]-1):
            out[buf][1:k+1]=np.ones(shape=(k))
            out2[buf][k+3:]=np.ones(shape=(512-k-2))
            out3[buf]=to_catigorical(k+1,512)
            buf+=1
    return out, out2, out3

In [4]:
model = get_m(
    token_num=3003,
    embed_dim=100,
    encoder_num=3,
    decoder_num=6,
    head_num=4,
    hidden_dim=120,
    attention_activation='relu',
    feed_forward_activation='relu',
    dropout_rate=0.05,
    embed_weights=np.random.random((3003, 100)),
    use_adapter=True,
)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
)
model.summary()
model.save('D:/decoder/models/dec_buf.h5')



Tensor("Shape:0", shape=(3,), dtype=int32)
Tensor("Shape_1:0", shape=(2,), dtype=int32)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




AssertionError: 

In [4]:
from keras.models import load_model
model = load_model ('D:/decoder/models/m5.h5' , custom_objects = get_custom_objects ())
print(K.get_value(model.optimizer.lr))
K.set_value(model.optimizer.lr, .0002)
print(K.get_value(model.optimizer.lr))




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


0.001
0.0002


In [5]:
from scipy.ndimage.interpolation import shift
n=0
e=1
bo = True

for i in range(1000):
    vectors_train = np.load('D:/decoder/data/data3000/news_vectors'+str(i)+'.npy')
    pieces_train = np.load('D:/decoder/data/data3000/news_pieces'+str(i)+'.npy')
    shifting = np.ones(shape=(pieces_train.shape[0],512))
    pieces_train = pieces_train+shifting
    
    print('Readed')  
        
    for j in range(5):
        ptt, kol = piecesextender(pieces_train[0+j*100:(1+j)*100])
        pot = np.copy(ptt)
        pot = shift(pot, (0,-1))
        pot = pot.reshape(pot.shape[0],512,1)
        vtt = vectorsextender(vectors_train[0+j*100:(1+j)*100], kol)
        nt, tt = kolextender(kol)
        
        
        model.fit(
            x=[vtt,
               nt,
               tt,
               ptt],
            y = pot,
            epochs=e, initial_epoch=n,
            validation_split=0.05,
            batch_size=6
        )
        n+=1
        e+=1

    if i%5==0 and bo:
        model.save('D:/decoder/models/m'+str(i)+'.h5')

Readed
Train on 5453 samples, validate on 287 samples
Epoch 1/1
5453/5453 [==============================] - 433s 79ms/step - loss: 7.1631 - val_loss: 3.8508
Train on 4615 samples, validate on 243 samples
Epoch 2/2
4615/4615 [==============================] - 355s 77ms/step - loss: 5.7817 - val_loss: 3.7864
Train on 4569 samples, validate on 241 samples
Epoch 3/3
4569/4569 [==============================] - 351s 77ms/step - loss: 5.5983 - val_loss: 3.4127
Train on 5488 samples, validate on 289 samples
Epoch 4/4
5488/5488 [==============================] - 423s 77ms/step - loss: 5.6053 - val_loss: 3.6657
Train on 4372 samples, validate on 231 samples
Epoch 5/5
4372/4372 [==============================] - 337s 77ms/step - loss: 5.2134 - val_loss: 3.1306
Readed
Train on 5344 samples, validate on 282 samples
Epoch 6/6
5344/5344 [==============================] - 412s 77ms/step - loss: 5.5773 - val_loss: 4.2312
Train on 5092 samples, validate on 268 samples
Epoch 7/7
5092/5092 [============

KeyboardInterrupt: 

In [7]:
model.save('D:/decoder/models/m4.h5')

In [ ]:
'''
def get_position_encoding(
    length, hidden_size, min_timescale=1.0, max_timescale=1.0e4):
  """Return positional encoding.
  Calculates the position encoding as a mix of sine and cosine functions with
  geometrically increasing wavelengths.
  Defined and formulized in Attention is All You Need, section 3.5.
  Args:
    length: Sequence length.
    hidden_size: Size of the
    min_timescale: Minimum scale that will be applied at each position
    max_timescale: Maximum scale that will be applied at each position
  Returns:
    Tensor with shape [length, hidden_size]
  """
  # We compute the positional encoding in float32 even if the model uses
  # float16, as many of the ops used, like log and exp, are numerically unstable
  # in float16.
  position = tf.cast(tf.range(length), tf.float32)
  num_timescales = hidden_size // 2
  log_timescale_increment = (
      math.log(float(max_timescale) / float(min_timescale)) /
      (tf.cast(num_timescales, tf.float32) - 1))
  inv_timescales = min_timescale * tf.exp(
      tf.cast(tf.range(num_timescales), tf.float32) * -log_timescale_increment)
  scaled_time = tf.expand_dims(position, 1) * tf.expand_dims(inv_timescales, 0)
  signal = tf.concat([tf.sin(scaled_time), tf.cos(scaled_time)], axis=1)
  return signal
'''